# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-a52c425994-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Literary Movements and Divine Comedy explorative search") 

Consider the following exploratory information need:

> investigate the literary movements and the poets. In particular, compare some italian poets in terms of place of birth, number of literary works and education.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q40185`| The Divine Comedy  | node |





Also consider

```
wd:Q40185 ?p ?obj .
```

is the BGP to retrieve all **properties of The Divine Comedy**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:

1. Who wrote The Divine Comedy? (return the IRI and the label of the author)

2. Identify the BGP for poet

3. Identify the BGP for literary movement

4. Return the poets born in the same place of the author of The Divine Comedy who are born between 1300 and 1500 (1300 and 1500 included) (the result set must be a list of couples poets IRI and label).

5. Identify the BGP for literary work

6. Compare the author of The Divine Comedy with Giovanni Boccaccio. Return for these poets the number of literary work they wrote (the result set must be two triples -one for each author- author IRI, label and #literary work).

7. Consider italian poets, their place of birth (the city) and their university education. Return for each city the number of poets who born there and who was educated there (the result set must be a list of triples city IRI, label and #poets).

8. Return the italian poets who won the Nobel Prize in Literature (the result set must be poet IRI and label). 

## Task 1
Who wrote The Divine Comedy? (return the IRI and the label of the author)

In [3]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q40185 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2551'), ('name', 'used metre')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de Fran

Final query for this task

In [116]:
queryString = """
SELECT ?obj ?name
WHERE {

   # Get the author of The Divine Comedy
   wd:Q40185 wdt:P50 ?obj .
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1067'), ('name', 'Dante Alighieri')]
1


## Task 2
Identify the BGP for poet

In [4]:
queryString = """
SELECT ?obj ?name
WHERE {
   
   wd:Q1067 ?p ?obj .
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1398'), ('name', 'Virgil')]
[('obj', 'http://www.wikidata.org/entity/Q9438'), ('name', 'Thomas Aquinas')]
[('obj', 'http://www.wikidata.org/entity/Q24633474'), ('name', 'audio podcast')]
[('obj', 'http://www.wikidata.org/entity/Q728592'), ('name', "Cecco d'Ascoli")]
[('obj', 'http://www.wikidata.org/entity/Q18084'), ('name', 'Vita Nuova')]
[('obj', 'http://www.wikidata.org/entity/Q102851'), ('name', 'Boethius')]
[('obj', 'http://www.wikidata.org/entity/Q12144794'), ('name', 'prosaist')]
[('obj', 'http://www.wikidata.org/entity/Q148540'), ('name', 'Republic of Florence')]
[('obj', 'http://www.wikidata.org/entity/Q15994177'), ('name', 'political theorist')]
[('obj', 'http://www.wikidata.org/entity/Q1700481'), ('name', 'Minneapolis Institute of Art')]
[('obj', 'http://www.wikidata.org/entity/Q1841'), ('name', 'Catholicism')]
[('obj', 'http://www.wikidata.org/entity/Q19180675'), ('name', 'Small Brockhaus and Efron Encyclopedic Dictionary')]

Final query for this task

In [112]:
queryString = """
SELECT ?p ?property
WHERE {
   
   #BGP for poet
   wd:Q49757 ?p ?obj .
   ?p sc:name ?property.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('property', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('property', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('property', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('property', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('property', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('property', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('property', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('property', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('property', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('property', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), 

## Task 3
Identify the BGP for literary movement

In [56]:
queryString = """
SELECT ?p ?obj ?name
WHERE {

  #wd:Q1067 ?p ?o.
  #?p sc:name ?name.
  
  wd:Q1067 wdt:P135 ?o.
  ?o ?p ?obj.
  ?obj sc:name ?name.
   
} ORDER BY ?year
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('obj', 'http://www.wikidata.org/entity/Q3326717'), ('name', 'literary movement')]
2


Final query for this task

In [114]:
queryString = """
SELECT ?p ?property
WHERE {

  #BGP for literary movement
  wd:Q3326717 ?p ?obj.
  ?p sc:name ?property.
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('property', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('property', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('property', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('property', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('property', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3553'), ('property', 'Zhihu topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('property', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('property', "topic's main category")]
8


## Task 4
Return the poets born in the same place of the author of The Divine Comedy who are born between 1300 and 1500 (1300 and 1500 included) (the result set must be a list of couples poets IRI and label).

In [105]:
queryString = """
SELECT ?s ?poet
WHERE {

   ?s wdt:P106 wd:Q49757.
   ?s sc:name ?poet.
   
   ?s wdt:P27 ?o1.
   ?o1 sc:name ?country1.
   
   ?s wdt:P569 ?born.
   
   BIND(STRBEFORE(str(?born), "-") AS ?year)
   
   wd:Q1067 wdt:P27 ?o .
   ?o sc:name ?country.
   
   FILTER(?country1 = ?country).
   FILTER(str(?year)>= "1300" && str(?year)<= "1500").
   
} 
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q62055718'), ('poet', "Lippo Pasci de' Bardi"), ('year', '1300')]
[('s', 'http://www.wikidata.org/entity/Q1149308'), ('poet', 'Rustico Filippi'), ('year', '1300')]
[('s', 'http://www.wikidata.org/entity/Q3827021'), ('poet', 'Lapo Gianni'), ('year', '1300')]
[('s', 'http://www.wikidata.org/entity/Q1402'), ('poet', 'Giovanni Boccaccio'), ('year', '1313')]
[('s', 'http://www.wikidata.org/entity/Q50399164'), ('poet', "Dolcibene de' Tori"), ('year', '1332')]
[('s', 'http://www.wikidata.org/entity/Q659093'), ('poet', 'Benedetto Dei'), ('year', '1418')]
[('s', 'http://www.wikidata.org/entity/Q1113645'), ('poet', 'Lucrezia Tornabuoni'), ('year', '1427')]
[('s', 'http://www.wikidata.org/entity/Q192374'), ('poet', 'Marsilio Ficino'), ('year', '1433')]
[('s', 'http://www.wikidata.org/entity/Q13570581'), ('poet', 'Bartolomeo della Fonte'), ('year', '1445')]
[('s', 'http://www.wikidata.org/entity/Q13570581'), ('poet', 'Bartolomeo della Fonte'), ('year'

Final query for this task

In [119]:
queryString = """
SELECT ?s ?poet ?year
WHERE {

   # Get the name of all poets by occupation
   ?s wdt:P106 wd:Q49757.
   ?s sc:name ?poet.
   
   # Get the citizenship of all poets
   ?s wdt:P27 ?o1.
   ?o1 sc:name ?country1.
   
   # Get the date of birth year of all poets
   ?s wdt:P569 ?born.
   
   BIND(STRBEFORE(str(?born), "-") AS ?year)
   
   # Get the citizenship of the author of The Divine Comedy
   wd:Q1067 wdt:P27 ?o .
   ?o sc:name ?country.
   
   # Filter by country and year
   FILTER(?country1 = ?country).
   FILTER(str(?year)>= "1300" && str(?year)<= "1500").
   
} ORDER BY ?year
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
The operation failed timed out


## Task 5
Identify the BGP for literary work

In [46]:
queryString = """
SELECT ?p ?name ?obj ?name2
WHERE {

   wd:Q40185 ?p ?obj.
   ?p sc:name ?name1.
   ?obj sc:name ?name2.
   
} 
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P674'), ('obj', 'http://www.wikidata.org/entity/Q754686'), ('name2', 'Cacus')]
[('p', 'http://www.wikidata.org/prop/direct/P674'), ('obj', 'http://www.wikidata.org/entity/Q258369'), ('name2', 'Peter Damian')]
[('p', 'http://www.wikidata.org/prop/direct/P674'), ('obj', 'http://www.wikidata.org/entity/Q706003'), ('name2', 'Ugolino della Gherardesca')]
[('p', 'http://www.wikidata.org/prop/direct/P674'), ('obj', 'http://www.wikidata.org/entity/Q129866'), ('name2', 'Minotaur')]
[('p', 'http://www.wikidata.org/prop/direct/P674'), ('obj', 'http://www.wikidata.org/entity/Q647496'), ('name2', 'Linus')]
[('p', 'http://www.wikidata.org/prop/direct/P674'), ('obj', 'http://www.wikidata.org/entity/Q272006'), ('name2', 'Antaeus')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('obj', 'http://www.wikidata.org/entity/Q2713307'), ('name2', 'Paradiso')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('obj', 'http://www.wikidata.org/entity/Q45

Final query for this task

In [115]:
queryString = """
SELECT ?p ?property
WHERE {

  #BGP for literary work
  wd:Q7725634 ?p ?obj.
  ?p sc:name ?property.
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('property', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('property', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('property', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('property', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('property', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('property', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('property', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('property', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('property', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('property', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('property', 'properties for t

## Task 6
Compare the author of The Divine Comedy with Giovanni Boccaccio. Return for these poets the number of literary work they wrote (the result set must be two triples -one for each author- author IRI, label and #literary work).

In [73]:
queryString = """
SELECT ?p ?name
WHERE {

   wd:Q8351202 ?p ?o.
   ?p sc:name ?name.
   

} 
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P110'), ('name', 'illustrator')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('name', 'part of the series')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P407'), ('name', 'language of work or name')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('name', 'country of origin')]
[('p', 'http://www.wikidata.org/prop/direct/P50'), ('name', 'author')]
[('p', 'http://www.wikidata.org/prop/direct/P577'), ('name', 'publication date')]
[('p', 'http://www.wikidata.org/prop/direct/P840'), ('name', 'narrative location')]
10


Final query for this task

In [71]:
queryString = """
SELECT ?name ?author (count(?work) AS ?literaryWork) 
WHERE {

   # Get all the literary work 
   ?s wdt:P31 wd:Q7725634.
   ?s sc:name ?work.
   
   # Get the author of all the literary work
   ?s wdt:P50 ?name.
   ?name sc:name ?author.
   
   # Filter the authors
   FILTER(str(?author) = "Giovanni Boccaccio" || str(?author) = "Dante Alighieri").
   
} GROUP BY ?name ?author
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'http://www.wikidata.org/entity/Q1402'), ('author', 'Giovanni Boccaccio'), ('literaryWork', '9')]
[('name', 'http://www.wikidata.org/entity/Q1067'), ('author', 'Dante Alighieri'), ('literaryWork', '12')]
2


## Task 7
Consider italian poets, their place of birth (the city) and their university education. Return for each city the number of poets who born there and who was educated there (the result set must be a list of triples city IRI, label and #poets).

In [93]:
queryString = """
SELECT ?place ?city (count(?poet) AS ?numPoet)
WHERE {

   ?poet wdt:P106 wd:Q49757.
   ?poet sc:name ?poetName.
   
   ?poet wdt:P106 ?occu.
   ?occu sc:name ?occup.
   
   ?poet wdt:P19 ?place.
   ?place sc:name ?city.
   
   ?poet wdt:P27 ?obj.
   ?obj sc:name ?country.
   
   ?poet wdt:P69 ?edu.
   ?edu sc:name ?uni.
   
   
   
   FILTER REGEX(str(?country), "^Italy").
   
} GROUP BY ?place ?city
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('place', 'http://www.wikidata.org/entity/Q13367'), ('city', 'Forlì'), ('numPoet', '11')]
[('place', 'http://www.wikidata.org/entity/Q220'), ('city', 'Rome'), ('numPoet', '79')]
[('place', 'http://www.wikidata.org/entity/Q13395'), ('city', 'Viterbo'), ('numPoet', '6')]
[('place', 'http://www.wikidata.org/entity/Q13396'), ('city', 'Rieti'), ('numPoet', '7')]
[('place', 'http://www.wikidata.org/entity/Q1449'), ('city', 'Genoa'), ('numPoet', '16')]
[('place', 'http://www.wikidata.org/entity/Q111954'), ('city', 'Santarcangelo di Romagna'), ('numPoet', '7')]
[('place', 'http://www.wikidata.org/entity/Q54650'), ('city', 'San Luca'), ('numPoet', '9')]
[('place', 'http://www.wikidata.org/entity/Q2634'), ('city', 'Naples'), ('numPoet', '5')]
[('place', 'http://www.wikidata.org/entity/Q39981'), ('city', 'Voghera'), ('numPoet', '3')]
[('place', 'http://www.wikidata.org/entity/Q284040'), ('city', 'Guasila'), ('numPoet', '6')]
[('place', 'http://www.wikidata.org/entity/Q391035'), ('city', 

Final query for this task

In [108]:
queryString = """
SELECT ?place ?city (count(?poet) AS ?numPoet)
WHERE {

   # Get all the poets
   ?poet wdt:P106 wd:Q49757.
   ?poet sc:name ?poetName.
   
   # Get the occupation 
   ?poet wdt:P106 ?occu.
   ?occu sc:name ?occup.
   
   # Get the place of birth of all poets
   ?poet wdt:P19 ?place.
   ?place sc:name ?city.
   
   # Get the country of citizenship 
   ?poet wdt:P27 ?obj.
   ?obj sc:name ?country.
   
    Get the educational institution
   ?poet wdt:P69 ?edu.
   ?edu sc:name ?uni.
   
   
   FILTER REGEX(str(?country), "^Italy").
   
} GROUP BY ?place ?city
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('place', 'http://www.wikidata.org/entity/Q6606'), ('city', 'Pordenone'), ('numPoet', '2')]
[('place', 'http://www.wikidata.org/entity/Q13367'), ('city', 'Forlì'), ('numPoet', '11')]
[('place', 'http://www.wikidata.org/entity/Q220'), ('city', 'Rome'), ('numPoet', '79')]
[('place', 'http://www.wikidata.org/entity/Q13395'), ('city', 'Viterbo'), ('numPoet', '6')]
[('place', 'http://www.wikidata.org/entity/Q13396'), ('city', 'Rieti'), ('numPoet', '7')]
[('place', 'http://www.wikidata.org/entity/Q1449'), ('city', 'Genoa'), ('numPoet', '16')]
[('place', 'http://www.wikidata.org/entity/Q111954'), ('city', 'Santarcangelo di Romagna'), ('numPoet', '7')]
[('place', 'http://www.wikidata.org/entity/Q54650'), ('city', 'San Luca'), ('numPoet', '9')]
[('place', 'http://www.wikidata.org/entity/Q2634'), ('city', 'Naples'), ('numPoet', '5')]
[('place', 'http://www.wikidata.org/entity/Q39981'), ('city', 'Voghera'), ('numPoet', '3')]
[('place', 'http://www.wikidata.org/entity/Q284040'), ('city', 

## Task 8
Return the italian poets who won the Nobel Prize in Literature (the result set must be poet IRI and label).

In [102]:
queryString = """
SELECT ?poetName ?prize
WHERE {

   ?poet wdt:P106 wd:Q49757.
   ?poet sc:name ?poetName.
   
   ?poet wdt:P27 ?obj.
   ?obj sc:name ?country.
   
   ?poet wdt:P166 ?award.
   ?award sc:name ?prize.
   
   
   FILTER REGEX(str(?country), "^Italy").
   FILTER REGEX(str(?prize), "^Nobel")
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('poetName', 'Dario Fo'), ('prize', 'Nobel Prize in Literature')]
[('poetName', 'Salvatore Quasimodo'), ('prize', 'Nobel Prize in Literature')]
[('poetName', 'Eugenio Montale'), ('prize', 'Nobel Prize in Literature')]
3


Final query for this task

In [111]:
queryString = """
SELECT ?poet ?poetName
WHERE {

   # Get all the poets
   ?poet wdt:P106 wd:Q49757.
   ?poet sc:name ?poetName.
   
   # get the citizenship of all poets
   ?poet wdt:P27 ?obj.
   ?obj sc:name ?country.
   
   # Get the award received by all poets
   ?poet wdt:P166 ?award.
   ?award sc:name ?prize.
   
   # Filter country and award
   FILTER REGEX(str(?country), "^Italy").
   FILTER REGEX(str(?prize), "^Nobel")
   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q765'), ('poetName', 'Dario Fo')]
[('poet', 'http://www.wikidata.org/entity/Q83038'), ('poetName', 'Salvatore Quasimodo')]
[('poet', 'http://www.wikidata.org/entity/Q83174'), ('poetName', 'Eugenio Montale')]
3
